# Task 2: [1] Implement a decision tree (DT) classifier by using one the information gain as the split criteria. 

####  Name: Yeo Zheng Xu Isaac

In [1]:
import pandas as pd
import numpy as np

# Read in the CSV file 
# convert all values "?" to null
df = pd.read_csv("C:/Users/Isaac Yeo/Downloads/magic04.data",
                  header=None, na_values="?" )
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,g
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,g
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,g
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,g
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,g


In [2]:
df[10].value_counts()

g    12332
h     6688
Name: 10, dtype: int64

In [3]:
cleanup_nums = {10: {"g": 0, "h": 1}}
df.replace(cleanup_nums, inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,28.7967,16.0021,2.6449,0.3918,0.1982,27.7004,22.0110,-8.2027,40.0920,81.8828,0
1,31.6036,11.7235,2.5185,0.5303,0.3773,26.2722,23.8238,-9.9574,6.3609,205.2610,0
2,162.0520,136.0310,4.0612,0.0374,0.0187,116.7410,-64.8580,-45.2160,76.9600,256.7880,0
3,23.8172,9.5728,2.3385,0.6147,0.3922,27.2107,-6.4633,-7.1513,10.4490,116.7370,0
4,75.1362,30.9205,3.1611,0.3168,0.1832,-5.5277,28.5525,21.8393,4.6480,356.4620,0


In [4]:
dt=np.array(df)
dt

array([[ 28.7967,  16.0021,   2.6449, ...,  40.092 ,  81.8828,   0.    ],
       [ 31.6036,  11.7235,   2.5185, ...,   6.3609, 205.261 ,   0.    ],
       [162.052 , 136.031 ,   4.0612, ...,  76.96  , 256.788 ,   0.    ],
       ...,
       [ 75.4455,  47.5305,   3.4483, ...,  30.2987, 256.5166,   1.    ],
       [120.5135,  76.9018,   3.9939, ...,  84.6874, 408.3166,   1.    ],
       [187.1814,  53.0014,   3.2093, ...,  52.731 , 272.3174,   1.    ]])

## Decision Tree Implementation as follows below:  

## STEPS & FUNCTIONS EXPLAINATION FOR DECISION TREE IMPLEMENTATION BELOW: 
### [1] Create Split:
#### Split refers to the attribute and the value that helps to divide the attribute index rows. This is achieved by finding gini index by 3 bits, breaking and evaluating the dataset.

### [2] Gini index:
#### To test the "split" dataset by separating the training patterns of two row groups and ini index help() to find the data set's gini index.

### [3] Split dataset:
#### This function breaks the dataset into two list of attribute and value indexed rows. After that, we can use the gini index to evaluate the cost of failed test split() by checking that of all attribute value that  belongs either to the left or right group.

### [4] Evaluating:
#### Use both above functions to find the best split, so then we can use it as a node. get split() is used to find the best split documented by iterating each attribute and values and returning them after checks are done.

### [5] Build Tree:
#### Build tree() function is to create the root node and the entire tree by calling the spilt() function which helps to use both the node as an argument and the max depth size of the node. 

In [25]:
from random import seed
from random import randrange
from csv import reader

### -- Split a dataset based on [1] attribute & [2] attribute value -- ###
def testing_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right

### -- Calculate Gini index for split dataset  -- ###
def gini_index(groups, classes):
   ### -- count all samples at split point  -- ###
    n_instances = float(sum([len(group) for group in groups]))
 ### -- sum weighted Gini index for each group  -- ###
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
      ### -- score the group based on the score for each class  -- ###
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
      ### -- weigh the group score by its relative size  -- ###
        gini += (1.0 - score) * (size / n_instances)
    return gini

### -- Select the best split point for a dataset  -- ### 
def getData_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = testing_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}
 
### -- Create a terminal node value  -- ###
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)
 
### -- Create child splits for a node or make terminal  -- ###
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
  ### -- Code is to check for a no split  -- ###
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
  ### -- Code is to check for max depth  -- ###
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
 ### -- To process left child  -- ###
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = getData_split(left)
        split(node['left'], max_depth, min_size, depth+1)
  ### -- To process right child  -- ###
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = getData_split(right)
        split(node['right'], max_depth, min_size, depth+1)
 
# Build a decision tree
def build_tree(train, max_depth, min_size):
    root = getData_split(train)
    split(root, max_depth, min_size, 1)
    return root

### -- Print function for decision tree  -- ###
def print_decision_tree(node, depth=0):
    if isinstance(node, dict):
        print('%s[X%d < %.3f]' % ((depth*' ', (node['index']+1), node['value'])))
        print_tree(node['left'], depth+1)
        print_tree(node['right'], depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

In [28]:
Deci_tree =build_tree(dt,10,1)
print_decision_tree(tree)

[X9 < 26.298]
 [X1 < 117.868]
  [X9 < 11.822]
   [X7 < -71.845]
    [X2 < 41.873]
     [X9 < 2.759]
      [X1 < 109.370]
       [X1 < 70.437]
        [0.0]
        [0.0]
       [X1 < 109.370]
        [0.0]
        [0.0]
      [X3 < 2.797]
       [X1 < 76.805]
        [1.0]
        [X1 < 89.596]
         [X1 < 76.805]
          [0.0]
          [0.0]
         [X1 < 89.596]
          [0.0]
          [0.0]
       [X4 < 0.290]
        [X10 < 271.107]
         [X4 < 0.256]
          [1.0]
          [0.0]
         [X7 < -83.503]
          [1.0]
          [0.0]
        [X1 < 90.939]
         [1.0]
         [X1 < 90.939]
          [1.0]
          [1.0]
     [X9 < 0.369]
      [0.0]
      [X8 < 53.439]
       [X1 < 98.811]
        [X1 < 85.750]
         [X1 < 82.474]
          [1.0]
          [1.0]
         [X1 < 85.750]
          [1.0]
          [1.0]
        [X1 < 98.811]
         [1.0]
         [1.0]
       [X1 < 115.325]
        [1.0]
        [0.0]
    [X5 < 0.298]
     [X2 < 57.592]
      [

# Task 2: [2] Implement a 10-fold cross-validation to evaluate the model.

## EXPLAINATION & STEPS FOR 10 FOLD CROSS-VALIDATION ON MODEL AS SHOWN BELOW:
### [1] We first call the Load csv() function to load the magic04dataset after converting all data string to index in the document. 

### [2] We then use the eval_algorithm() which then implements the 10 fold cross-validation.  Meaning, firstly the data will be split into 10 folds equally, followed by measuring the accuracy by using the calc_accuracy_metric() function.


In [27]:
#  from random import seed  -- ###
from random import randrange
from csv import reader
  
### -- Load a CSV file -- ###
def load_csv(filename):
    file = open(filename, "rt")
    lines = reader(file)
    dataset = list(lines)
    return dataset
 
### -- Convert string column to float -- ###
def str_column_to_float(dataset, column):
    for row in dataset:
        row[column] = float(row[column].strip())

### -- Function to split dataset into k folds -- ###
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split
 
### -- Calc accuracy percentage % -- ###
def calc_accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0
 
### -- Evaluate an algorithm using a cross validation split -- ###
def eval_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = calc_accuracy_metric(actual, predicted)
        scores.append(accuracy)
    return scores
 
### -- Split a dataset based on an attribute and an attribute value  -- ###
def testing_split(index, value, dataset):
    left, right = list(), list()
    for row in dataset:
        if row[index] < value:
            left.append(row)
        else:
            right.append(row)
    return left, right
 
### -- Calculate the Gini index for a split dataset  -- ###
def gini_index(groups, classes):
    n_instances = float(sum([len(group) for group in groups]))
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = 0.0
        for class_val in classes:
            p = [row[-1] for row in group].count(class_val) / size
            score += p * p
        gini += (1.0 - score) * (size / n_instances)
    return gini
 
### -- Select the best split point for a dataset -- ###
def getData_split(dataset):
    class_values = list(set(row[-1] for row in dataset))
    b_index, b_value, b_score, b_groups = 999, 999, 999, None
    for index in range(len(dataset[0])-1):
        for row in dataset:
            groups = testing_split(index, row[index], dataset)
            gini = gini_index(groups, class_values)
            if gini < b_score:
                b_index, b_value, b_score, b_groups = index, row[index], gini, groups
    return {'index':b_index, 'value':b_value, 'groups':b_groups}
 
### -- Create a terminal node value  -- ###
def to_terminal(group):
    outcomes = [row[-1] for row in group]
    return max(set(outcomes), key=outcomes.count)
 
### -- Create child splits for a node or make terminal  -- ###
def split(node, max_depth, min_size, depth):
    left, right = node['groups']
    del(node['groups'])
    if not left or not right:
        node['left'] = node['right'] = to_terminal(left + right)
        return
    if depth >= max_depth:
        node['left'], node['right'] = to_terminal(left), to_terminal(right)
        return
    if len(left) <= min_size:
        node['left'] = to_terminal(left)
    else:
        node['left'] = getData_split(left)
        split(node['left'], max_depth, min_size, depth+1)
    if len(right) <= min_size:
        node['right'] = to_terminal(right)
    else:
        node['right'] = getData_split(right)
        split(node['right'], max_depth, min_size, depth+1)
        
### --  Build Decision Tree  -- ###
def build_tree(train, max_depth, min_size):
    root = getData_split(train)
    split(root, max_depth, min_size, 1)
    return root
 
# Make a prediction with a decision tree  -- ###
def predict(node, row):
    if row[node['index']] < node['value']:
        if isinstance(node['left'], dict):
            return predict(node['left'], row)
        else:
            return node['left']
    else:
        if isinstance(node['right'], dict):
            return predict(node['right'], row)
        else:
            return node['right']      

### -- Classification & Regression Algorithm -- ###
def decision_tree(train, test, max_depth, min_size):
    tree = build_tree(train, max_depth, min_size)
    predictions = list()
    for row in test:
        prediction = predict(tree, row)
        predictions.append(prediction)
    return(predictions)
 

seed(1)
filename = 'C:/Users/Isaac Yeo/Downloads/magic04.data'
dataset = load_csv(filename)
### -- convert string attributes to integers  -- ###
for i in range(len(dataset[0])):
    str_column_to_float(dataset, i)
### -- evaluation algorithm  -- ###
n_folds = 10
max_depth = 5
min_size = 10
scores = eval_algorithm(dataset, decision_tree, n_folds, max_depth, min_size)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

Mean Accuracy: 97.445%
